In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# Завантаження тексту
path = "Franko_-Zibrannya-tvoriv-u-p-yatdesyati-tomah-literaturno-kritichni-praci-1900-1902-tom-33-.382254.txt"
with open(path, "r", encoding="utf-8") as f:
    text = f.read().lower()

# Використовуємо перші 100 000 символів для швидкого навчання
text = text[:100_000]
print("Довжина тексту:", len(text))

# Підготовка словника
chars = sorted(list(set(text)))
vocab_size = len(chars)
char_to_idx = {c: i for i, c in enumerate(chars)}
idx_to_char = {i: c for i, c in enumerate(chars)}
print("Розмір словника:", vocab_size)

SEQ_LEN = 90
STEP = 3

sequences, next_chars = [], []
for i in range(0, len(text) - SEQ_LEN, STEP):
    sequences.append(text[i:i + SEQ_LEN])
    next_chars.append(text[i + SEQ_LEN])


class TextDataset(Dataset):
    def __init__(self, sequences, next_chars):
        self.X = torch.tensor(
            [[char_to_idx[c] for c in seq] for seq in sequences], dtype=torch.long
        )
        self.y = torch.tensor([char_to_idx[c] for c in next_chars], dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


device = "cuda" if torch.cuda.is_available() else "cpu"
dataset = TextDataset(sequences, next_chars)
loader = DataLoader(dataset, batch_size=128, shuffle=True, pin_memory=(device == "cuda"))


# Модель GRU
class GRUGenerator(nn.Module):
    def __init__(self, vocab_size, embed_dim=64, hidden_dim=160):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_dim)
        self.gru = nn.GRU(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden=None):
        x = self.embed(x)
        out, hidden = self.gru(x, hidden)
        out = self.fc(out[:, -1])
        return out, hidden


model = GRUGenerator(vocab_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.002)
print(model)

# Навчання
EPOCHS = 8
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for X_batch, y_batch in tqdm(loader, desc=f"Epoch {epoch + 1}/{EPOCHS}"):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        preds, _ = model(X_batch)
        loss = criterion(preds, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}/{EPOCHS} — Loss: {total_loss / len(loader):.4f}")


# Топ-k + топ-p sampling
def sample_top_k_p(preds, k=5, p=0.9, temperature=0.7):
    preds = torch.log(preds + 1e-9) / temperature
    preds = torch.softmax(preds, dim=0)

    top_k_probs, top_k_idx = torch.topk(preds, k)
    top_k_probs = top_k_probs / top_k_probs.sum()

    sorted_probs, sorted_idx = torch.sort(top_k_probs, descending=True)
    cumulative_probs = torch.cumsum(sorted_probs, dim=0)
    cutoff = (cumulative_probs > p).nonzero(as_tuple=False)
    if len(cutoff) > 0:
        cutoff_idx = cutoff[0].item() + 1
        sorted_probs = sorted_probs[:cutoff_idx]
        sorted_idx = sorted_idx[:cutoff_idx]
        sorted_probs = sorted_probs / sorted_probs.sum()

    idx = torch.multinomial(sorted_probs, 1).item()
    return top_k_idx[sorted_idx[idx]].item()


# Генерація тексту
def generate_text(seed, length=500, temperature=0.7, k=5, p=0.9):
    model.eval()
    generated = seed
    seq = seed[-SEQ_LEN:].lower()
    hidden = None
    for _ in range(length):
        x = torch.tensor([[char_to_idx.get(c, 0) for c in seq]], dtype=torch.long).to(device)
        with torch.no_grad():
            preds, hidden = model(x, hidden)
        next_idx = sample_top_k_p(torch.softmax(preds[0], dim=0), k=k, p=p, temperature=temperature)
        next_char = idx_to_char[next_idx]
        generated += next_char
        seq = seq[1:] + next_char
    return generated


seed = ("література українська новела і поетичний стиль з глибокими образами "
        "та старовинними словами Франка ")

print(generate_text(seed, temperature=0.7, k=5, p=0.9))

Довжина тексту: 100000
Розмір словника: 102
GRUGenerator(
  (embed): Embedding(102, 64)
  (gru): GRU(64, 160, batch_first=True)
  (fc): Linear(in_features=160, out_features=102, bias=True)
)


Epoch 1/8: 100%|██████████| 261/261 [00:26<00:00,  9.71it/s]


Epoch 1/8 — Loss: 2.7638


Epoch 2/8: 100%|██████████| 261/261 [00:28<00:00,  9.12it/s]


Epoch 2/8 — Loss: 2.3148


...

Epoch 6/8 — Loss: 1.7738


Epoch 7/8: 100%|██████████| 261/261 [00:31<00:00,  8.23it/s]


Epoch 7/8 — Loss: 1.6828


Epoch 8/8: 100%|██████████| 261/261 [00:26<00:00,  9.76it/s]


Epoch 8/8 — Loss: 1.6018
література українська новела і поетичний стиль з глибокими образами та старовинними словами Франка посьманних автор по станні з неї, значенним до не попує в на сам з найського і наукова того, що бутовних і вірнув і з посного на польських україні (стор. 25). по полоського те, відбував з польські в насі на відповідним до такої і наші на напів і замановим насі на відповідання притики велість до не значній матеріальніших інтерестру з наші такі повідана станні польська тільки поворобів і він вистава старальних присвітськах осніальних і попівського і не мого літературних про насільних українській р
